# MACD

异同移动（Moving Average Convergence/Divergence）是由指数均线演变而来，是股票分析中常见的指标。

MACD指标，反应股票价格走势能量和变化强度，通过“2线1柱”发现股票的买卖点。

MACD，是由“2线1柱”组成：
- DIF 快速线 - 一般由12日指数加权平均值减去26日指数加权移动平均值
- DEA慢速线 - 是DIF的9日指数加权移动平均值。
- MACD 柱状图 - DIF 减去 DEA。
MACD默认的参数：12、26、9。

## MACD 计算公式

$$
\begin{aligned}
& DIF = EMA_{(close,12)} - EMA_{(close,26)} \\
& DEA = EMA_{(DIF,9)}\\
& MCAD = DIF - DEA
\end{aligned}

$$

- DIF ：12日指数移动平均值EMA12 减去 26日指数移动平均值EMA26
- DEA：计算DIF的9日指数移动平均值EMA9

## MACD应用说明
- DIF、DEA均为正，DIF向上突破DEA，买入信号；
- DIF、DEA均为负，DIF向下跌破DEA，卖出信号。

# MACD 交易策略

## 策略规则

MACD指标，反应股票价格走势能量和变化强度。

1. 计算 DIF、DEA、MACD指标
2. 计算买入卖出信号
- DIF、DEA均为正，DIF向上突破DEA，买入信号；
- DIF、DEA均为负，DIF向下跌破DEA，卖出信号。

在这里，601318 中国平安 为例，计算其2022-01-01 至 2023-01-01期间每日行情的MA5（短期均线）、MA20（长期均线）。


## 数据准备

In [2]:
import sys
sys.path.append(r'/Users/paul/DSWorkspace/001量化策略/002-Project项目/lleasy')

# 1. 数据准备
from lleasy.database import SqliteDatabase as sqlitedb
from lleasy.object import TradeData, BarData
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

In [48]:
# 获取数据
# 标的：601318 中国平安
# 时间：2022-01-01 至 2023-01-01
# 频率：日行情
#获取sqlite链接
db = sqlitedb()
datas= db.get_symbol_bars_bySEDate('601318','2022-01-01','2023-01-01')
datas=datas.set_index(['datetime'])

2023-08-03 15:57:23,948 DEBUG sqlalchemy.pool.impl.QueuePool Created new connection <sqlite3.Connection object at 0x125d0b940>
2023-08-03 15:57:23,949 DEBUG sqlalchemy.pool.impl.QueuePool Connection <sqlite3.Connection object at 0x125d0b940> checked out from pool
2023-08-03 15:57:23,949 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-03 15:57:23,951 INFO sqlalchemy.engine.Engine SELECT llbardata.id AS llbardata_id, llbardata.symbol AS llbardata_symbol, llbardata.interval AS llbardata_interval, llbardata.open AS llbardata_open, llbardata.close AS llbardata_close, llbardata.high AS llbardata_high, llbardata.low AS llbardata_low, llbardata.vol AS llbardata_vol, llbardata.turnover AS llbardata_turnover, llbardata.asset AS llbardata_asset, llbardata.adjust AS llbardata_adjust, llbardata.datetime AS llbardata_datetime 
FROM llbardata 
WHERE llbardata.symbol = ? AND llbardata.datetime >= ? AND llbardata.datetime <= ?
2023-08-03 15:57:23,952 INFO sqlalchemy.engine.Engine [generated in 0

## 编制策略

In [73]:
bars =datas
bars['close'] = bars['close'].astype('float')
ewma9_num = 9
ewma12_num = 12
ewma26_num = 26

#计算EWMA
#ratio 当前股价的权重
def ewma(data:pd.Series, period:int, ratio:float) -> pd.Series:
    data = data.astype(float)
    # 因为[]是从0开始计算,df.iloc[0:period]取值时，右括号值不取。。
    # 计算第period - 1 和 period的EWMA
    ewma = pd.Series(0.0,index=data.index)
    ewma_period = np.mean(data[0:period])
    ewma[period-1] = ewma_period
    
    for i in range(period,len(data)):
        ewma[i] = ewma[i-1] * (1 - ratio) + data[i] * ratio
    
    return ewma

#计算DIF、DEA、MACD
bars['ewma_12'] = ewma(bars['close'],12,2/(12+1))
bars['ewma_26'] = ewma(bars['close'],26,2/(26+1))

bars['DIF'] = bars['ewma_12'] - bars['ewma_26']
bars['DEA'] = ewma(bars['DIF'],9,2/(9+1))
bars['MACD'] = bars['DIF'] - bars['DEA']

## 执行策略

In [ ]:
#- DIF、DEA均为正，DIF向上突破DEA，买入信号；
#- DIF、DEA均为负，DIF向下跌破DEA，卖出信号。
bars['signal'] = np.where(bars['DIF'] > bars['DEA'],1,-1)
bars= bars.iloc[40:]
bars[['close','DIF','DEA','MACD','signal']]

In [67]:
# 图形化显示
import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = make_subplots(
    rows=2,cols=1,
    shared_xaxes=True,
)

trace1 = go.Scatter(
    x=bars.index,
    y=bars['DIF'],
    mode='lines',
    name='DIF',
    marker={
        'size':8,
#        'color':'rgba(102,198,147,0.7)',
    }
)
trace2 = go.Scatter(
    x=bars.index,
    y=bars['DEA'],
    mode='lines',
    name='DEA',
    marker={
        'size':8,
#        'color':'rgba(252,108,117,1)'
    }
)

trace3 = go.Scatter(
    x=bars.index,
    y=bars['close'],
    mode='lines',
    name='close',
    marker={
        'size':8,
#        'color':'rgba(252,108,117,1)'
    }
)

trace4 = go.Bar(
    x=bars.index,
    y=bars['MACD'],
    name='k线'
)

fig.add_traces(
    trace1,
    rows=2,cols=1
)
fig.add_traces(
    trace2,
    rows=2,cols=1
)
fig.add_traces(
    trace4,
    rows=2,cols=1
)
fig.add_traces(
    trace3,
    rows=1,cols=1
)

#fig = go.Figure(data=[trace1,trace2,trace3,trace4])

fig.update_xaxes(
        # 在x轴上，去除周六、周日
    rangebreaks=[
        dict(bounds=['sat','mon']),
    ]
)

fig.update_layout(
    hovermode="x"
)

fig.show()

## 策略回测

In [77]:
# 计算收益率
bars['rtn'] = bars['close'] / bars['close'].shift(1) - 1
# 计算对数收益率
bars['ln_rtn'] = np.log(bars['close'] / bars['close'].shift(1))
# 计算策略收益率
bars['stragey_rtn'] = bars['signal'].shift(1) * bars['ln_rtn']
bars.dropna(inplace=True)

np.exp(bars[['ln_rtn','stragey_rtn']]).sum()

np.exp(bars[['ln_rtn','stragey_rtn']]).std()*252**0.5


/var/folders/d9/rbb57jqn3g133_ynk3vgz02c0000gn/T/ipykernel_10724/2995402561.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/d9/rbb57jqn3g133_ynk3vgz02c0000gn/T/ipykernel_10724/2995402561.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/d9/rbb57jqn3g133_ynk3vgz02c0000gn/T/ipykernel_10724/2995402561.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

ln_rtn         0.276892
stragey_rtn    0.273356
dtype: float64


# 总结